## Prerequisite

In [ ]:
import ipywidgets as widgets

## Datasets

In [ ]:
from anki_ai_helper.dataset.german_word_list import GermanWordList

word_list = GermanWordList()

for row in word_list:
    print(row)
    break

## Text Generator

### Vicuna

In [ ]:
import torch

from anki_ai_helper.LLM.vicuna_7b_v15 import Vicuna7Bv15

SYSTEM_PROMPT = "As a language model assisting with German language learning, provide precise explanations, examples, and translations for various German words. Focus on delivering responses in a structured, concise format, without adding any extra information or notes."
QUERY_PROMPT = "Provide the singular and plural forms of the German noun 'der Apfel', with their translations in parentheses. Use 'Singular:' for the singular form with its article, and 'Plural:' for the plural form with its article."

with torch.cuda.amp.autocast(dtype=torch.bfloat16):
    with Vicuna7Bv15(SYSTEM_PROMPT) as vicuna:
        print(vicuna.shoot(QUERY_PROMPT))

### Mistral

In [ ]:
import torch

from anki_ai_helper.LLM.mistral_7b_instruct_v02 import Mistral7BInstructV02

SYSTEM_PROMPT = "As a language model assisting with German language learning, provide precise explanations, examples, and translations for various German words. Focus on delivering responses in a structured, concise format, without adding any extra information or notes."
QUERY_PROMPT = "Provide the singular and plural forms of the German noun 'der Apfel', with their translations in parentheses. Use 'Singular:' for the singular form with its article, and 'Plural:' for the plural form with its article."

with torch.cuda.amp.autocast(dtype=torch.bfloat16):
    with Mistral7BInstructV02(SYSTEM_PROMPT) as vicuna:
        print(vicuna.shoot(QUERY_PROMPT))

## Text to Speech

In [ ]:
from datetime import datetime

from anki_ai_helper.T2S.tts_v2 import TTSV2
from anki_ai_helper.helper import io as io_utils

TEMP_DIR = io_utils.create_temp_directory(f'text_to_speech')
T2S_OUTPUT_FILENAME = f"TTS_{datetime.now().strftime('%y%m%d-%H%M%S')}"
TEXT = "A curious fox named Finn discovered a hidden grove in the enchanted forest, where fireflies danced around a crystal pond, whispering tales of starlit secrets."

with TTSV2('en', TEMP_DIR) as tts:
    tts.shoot(TEXT, T2S_OUTPUT_FILENAME)

## Anki Deck

In [ ]:
from anki_ai_helper.anki.style.two_sentence_puzzler import TwoSentencePuzzlerStyle, TwoSentencePuzzlerFields, TwoSentencePuzzlerNote
from anki_ai_helper.anki.deck import AnkiDeck

from anki_ai_helper.helper import io as io_helper

deck_style = TwoSentencePuzzlerStyle()
deck_fields = TwoSentencePuzzlerFields()
deck = AnkiDeck(deck_name='example', style=deck_style, fields=deck_fields)
note = TwoSentencePuzzlerNote({
    "word": 'sehr',
    "word_trans": 'very',
    "1_pzl": 'Das Wetter heute ist .... schön. (Hint: An adverb to enhance \'schön\')',
    "1_fil": 'Das Wetter heute ist sehr schön.',
    "1_trans": 'The weather today is very nice.',
    "1_pzl_vce": '',
    "1_fil_vce": '',
    "1_trans_vce": '',
    "2_pzl": 'Ich möchte ..... Kaffee trinken. (Hint: An indefinite article for \'Kaffee\')',
    "2_fil": 'Ich möchte einen Kaffee trinken.',
    "2_trans": 'I would like to drink a coffee.',
    "2_pzl_vce": '',
    "2_fil_vce": '',
    "2_trans_vce": '',
    "expl_1": 'The word \'sehr\' enhances adjectives or adverbs, similar to \'very\' in English.',
    "expl_2": '\'Einen\' is the accusative form of the indefinite article \'ein\' used for masculine nouns like \'Kaffee\'.',
})

deck.add_note(note)

deck_path = deck.save([])
assert deck_path != None

n_deleted_files = io_helper.delete_file(deck_path)
assert n_deleted_files == 1

## Text to Image

In [ ]:
from IPython.display import Image
from anki_ai_helper.T2I.stable_diffusion_v15 import StableDiffusionV15

prompt = 'A tiny, fluffy kitten in oversized glasses and a lab coat.'

with StableDiffusionV15() as t2i:
    image, image_path = t2i.run(prompt=prompt, filename='cute_kitten')
    display(Image(filename=image_path))

    n_deleted_files = io_helper.delete_file(image_path)
    assert n_deleted_files == 1

## Anki Styles

### AI-SprachMeister (CSV File)

In [ ]:
from anki_ai_helper.LLM.mistral_7b_instruct_v02 import Mistral7BInstructV02
from anki_ai_helper.anki.ai_sprach_meister import AiSprachMeister
from anki_ai_helper.dataset.german_word_list import GermanWordList

word_list = GermanWordList(f=0, t=1100)
name = "AI-SprachMeister: 4000 German Words in Context"

anki_deck = AiSprachMeister(Mistral7BInstructV02, word_list, name)

anki_deck.generate_sentences()
anki_deck.fetch_extra_info()
anki_deck.to_voice()
anki_deck.convert_to_mp3()
anki_deck.package_deck(1000, n_decks=1)

AI-SprachMeister (Dictionary)

In [ ]:
from anki_ai_helper.LLM.mistral_7b_instruct_v02 import Mistral7BInstructV02
from anki_ai_helper.anki.ai_sprach_meister import AiSprachMeister
from anki_ai_helper.dataset.dict_word_list import DictWordList

NOUN_TYPE = "Noun"
VERB_TYPE = "Verb"
OTHER_TYPE = "Other"

entries = {
    'gehen': VERB_TYPE,
    'der Apfel': NOUN_TYPE,
    'respektvoll': OTHER_TYPE,
}
word_list = DictWordList(entries)
name = "AI-SprachMeister (my words)"

anki_deck = AiSprachMeister(Mistral7BInstructV02, word_list, name)
anki_deck.generate_sentences()
anki_deck.fetch_extra_info()
anki_deck.to_voice()
anki_deck.convert_to_mp3()
anki_deck.package_deck(1000)